<a href="https://colab.research.google.com/github/eiziiaizii1/ceng442-assignment1-GroupTAFB/blob/main/Notebook_ceng442_assignment1_GroupTAFB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Github - Google Colab test